# **Feature engineering of the Olist database:**<br/>*Standardization of geolocations*

Geolocation is a fundamental aspect of datasets, as it provides valuable information about territoriality. The dataset we are working with includes a table that associates administrative locations (such as state, municipality, and zip code) with their physical GPS coordinates. Additionally, the customer and vendor tables in the dataset only contain administrative location information.

However, during our exploration of the relational schema, we discovered that the city names have not been standardized, causing multiple spellings of the same city. The territory is a key feature that relates to solvency and socioeconomic factors, and it is crucial to cleanse and standardize the city names in order to use them effectively. This notebook focuses on addressing this issue by utilizing online resources such as Wikipedia and open data CSV files to correct and standardize the names of municipalities.

# Problem demonstration

## Geolocations table

In [37]:
from olist_commons import get_raw_geolocations
from pepper_commons import discrete_stats
raw_geolocations = get_raw_geolocations()
display(discrete_stats(raw_geolocations, 'geolocation'))
raw_geolocations_scz_keys = raw_geolocations[[
    'geolocation_state',
    'geolocation_city',
    'geolocation_zip_code_prefix'
]].copy().drop_duplicates()
raw_geolocations_scz_keys.columns = ['state', 'city', 'zip_code']
display(raw_geolocations_scz_keys)

,n,n_u,n_na,Filling rate,Shannon entropy,dtypes
geolocation,,,,,,
geolocation_zip_code_prefix,1000163,19015,0,1.0,0.019012,object
geolocation_lat,1000163,717372,0,1.0,0.717255,object
geolocation_lng,1000163,717615,0,1.0,0.717498,object
geolocation_city,1000163,8011,0,1.0,0.008010,object
geolocation_state,1000163,27,0,1.0,0.000027,object


,state,city,zip_code
0,SP,sao paulo,01037
1,SP,sao paulo,01046
3,SP,sao paulo,01041
4,SP,sao paulo,01035
5,SP,são paulo,01012
...,...,...,...
999806,RS,ibiaçá,99940
999846,RS,santa cecilia do sul,99952
999867,RS,ciríaco,99970
999891,RS,estação,99930


## The postal code is not a primary key for municipalities


**19015** postal codes but **19023** municipalities.

In [38]:
print(raw_geolocations_scz_keys.zip_code.unique().shape[0])

19015


## Is a code associated with a single municipality and state?

Some postal codes are associated with up to 5 names of municipalities.

For example, Emby-Guaçu is spelled in 5 different ways.

In [43]:
display(
    raw_geolocations_scz_keys
    .groupby(by='zip_code')
    .agg(['count', tuple])
    .sort_values(by=('city', 'count'), ascending=False)
)

state                        city  \
         count                 tuple count   
zip_code                                     
13457        5  (SP, SP, SP, SP, SP)     5   
78290        5  (MT, MT, MT, MT, MT)     5   
13455        5  (SP, SP, SP, SP, SP)     5   
13454        5  (SP, SP, SP, SP, SP)     5   
42850        5  (BA, BA, BA, BA, BA)     5   
...        ...                   ...   ...   
38444        1                 (MG,)     1   
38445        1                 (MG,)     1   
38446        1                 (MG,)     1   
38447        1                 (MG,)     1   
99990        1                 (RS,)     1   

                                                             
                                                      tuple  
zip_code                                                     
13457     (santa bárbara d'oeste, santa barbara d oeste,...  
78290     (figueiropolis d'oeste, figueiropolis d oeste,...  
13455     (santa barbara d'oeste, santa barbara d oeste,...  
13454     (santa barbara d'oeste, santa bárbara d'oeste,...  
42850     (dias d'avila, dias d'ávila, dias davila, dias...  
...                                                     ...  
38444                                           (araguari,)  
38445                                           (araguari,)  
38446                                           (araguari,)  
38447                                           (araguari,)  
99990                                          (muliterno,)  

[19015 rows x 4 columns]

In [16]:
display(raw_geolocations_scz_keys[raw_geolocations_scz_keys.zip_code == '06900'])

,state,city,zip_code
177377,SP,embu-guacu,06900
177379,SP,embu-guaçu,06900
177382,SP,embu guaçu,06900
177423,SP,embu guacu,06900
177502,SP,embuguacu,06900


# Sizing of the problem

We first retrieve the raw data from the 3 tables concerned, then for each of them, we reduce the table to only unique ZCS `(zip, city, state)` triplets, which allows us to perform the following counts:
* **8 011** municipalities in `geolocations` table, while there are only **5 569** municipalities in Brazil.
* **4 119** municipalities in `customer_orders` table
* **611** municipalities in `sellers` table

In [32]:
from pepper_commons import discrete_stats
from olist_commons import (
    get_raw_customers,
    get_raw_sellers,
    get_raw_geolocations,
    remove_columns_prefixes,
    get_zcs_reduction
)
raw_customer_orders = get_raw_customers()
raw_sellers = get_raw_sellers()
raw_geolocations = get_raw_geolocations()

In [33]:
remove_columns_prefixes(raw_geolocations, 'geolocation_')
display(discrete_stats(get_zcs_reduction(raw_geolocations)))

,n,n_u,n_na,Filling rate,Shannon entropy,dtypes
geolocation,,,,,,
zip_code_prefix,27912,19015,0,1.0,0.681248,object
city,27912,8011,0,1.0,0.287009,object
state,27912,27,0,1.0,0.000967,object


In [34]:
remove_columns_prefixes(raw_customer_orders, 'customer_')
display(discrete_stats(get_zcs_reduction(raw_customer_orders)))

,n,n_u,n_na,Filling rate,Shannon entropy,dtypes
customers,,,,,,
zip_code_prefix,15034,14994,0,1.0,0.997339,object
city,15034,4119,0,1.0,0.273979,object
state,15034,27,0,1.0,0.001796,object


In [35]:
remove_columns_prefixes(raw_sellers, 'seller_')
display(discrete_stats(get_zcs_reduction(raw_sellers)))

,n,n_u,n_na,Filling rate,Shannon entropy,dtypes
sellers,,,,,,
zip_code_prefix,2296,2246,0,1.0,0.978223,object
city,2296,611,0,1.0,0.266115,object
state,2296,23,0,1.0,0.010017,object
